# Implementa e testa um Multi Layer Perceptron (MLP)

Importa as bibliotecas utilizadas:

In [4]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


Cria a classe MLP que implementa um algoritmo de Multi Layer Perceptron usando Numpy:

In [2]:
class MLP:
    def __init__(self, layer_sizes, activations):
        self.layer_sizes = layer_sizes
        self.activations = activations
        self.num_layers = len(layer_sizes)
        
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i-1]) * np.sqrt(2 / layer_sizes[i-1]) for i in range(1, self.num_layers)]
        self.biases = [np.zeros((layer_sizes[i], 1)) for i in range(1, self.num_layers)]
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def softmax(self, x):
        exp_values = np.exp(x - np.max(x, axis=0, keepdims=True))
        return exp_values / np.sum(exp_values, axis=0, keepdims=True)
    
    def forward(self, x):
        activations = [x]
        for i in range(self.num_layers - 1):
            z = np.dot(self.weights[i], activations[-1]) + self.biases[i]
            activation_fn = getattr(self, self.activations[i])
            activation = activation_fn(z)
            activations.append(activation)
        return activations
    
    def backward(self, x, y, lr):
        m = x.shape[1]
        activations = self.forward(x)
        deltas = [None] * (self.num_layers - 1)
        deltas[-1] = activations[-1] - y
        
        for i in range(self.num_layers - 2, 0, -1):
            delta = np.dot(self.weights[i].T, deltas[i]) * (activations[i] > 0)
            deltas[i - 1] = delta
 
        for i in range(self.num_layers - 1):
            self.weights[i] -= lr * np.dot(deltas[i], activations[i].T) / m
            self.biases[i] -= lr * np.sum(deltas[i], axis=1, keepdims=True) / m
    
    def train(self, x_train, y_train, epochs, lr):
        for epoch in range(epochs):
            self.backward(x_train, y_train, lr)
            if epoch % 100 == 0:
                activations = self.forward(x_train)
                loss = -np.mean(np.sum(y_train * np.log(activations[-1]), axis=0))
                print(f'Epoch {epoch}, Loss: {loss}')
    
    def predict(self, x_test):
        activations = self.forward(x_test)
        return np.argmax(activations[-1], axis=0)


testando com um pequeno dataset de treino:

In [3]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
Y = np.array([[1, 0], [0, 1], [0, 1], [1, 0]]).T

layer_sizes = [2, 64, 64, 2]  
activations = ['relu', 'relu', 'softmax']  

mlp = MLP(layer_sizes, activations)
mlp.train(X, Y, epochs=1000, lr=0.01)

predictions = mlp.predict(X)
print("Predições:", predictions)


Epoch 0, Loss: 1.6580873506718343
Epoch 100, Loss: 0.2526273821304806
Epoch 200, Loss: 0.15918994087711968
Epoch 300, Loss: 0.11268314361324461
Epoch 400, Loss: 0.08416234704459528
Epoch 500, Loss: 0.06502278719214341
Epoch 600, Loss: 0.05165771418437252
Epoch 700, Loss: 0.041961416636495746
Epoch 800, Loss: 0.03475912921244856
Epoch 900, Loss: 0.029303511690233406
Predições: [0 1 1 0]


Teste Simples usando a MLPClassifier do Sklearn:

In [4]:
mlp = MLPClassifier(hidden_layer_sizes=(64, 64), activation='relu', solver='adam', max_iter=1000, random_state=42)
mlp.fit(X.T, np.argmax(Y, axis=0))  

predictions = mlp.predict(X.T)

print("Predições:", predictions)
accuracy = accuracy_score(np.argmax(Y, axis=0), predictions)
print("Acurácia:", accuracy)

Predições: [0 1 1 0]
Acurácia: 1.0
